In [ ]:
# default_exp secret
# hide

_FNAME='secret'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os

TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
HERE = os.path.abspath('')

In [ ]:
import boto3
CLIENT = boto3.client('secretsmanager')

In [ ]:
def get(name, client=None, **kwargs):
    '''
    Retrieves the secret with the given name
    '''
    client = client or CLIENT
    resp = client.get_secret_value(SecretId=name, **kwargs)
    str_value = resp.get('SecretString')
    byte_value = resp.get('SecretBytes')
    return str_value or byte_value

def create(name, value, client=None, **kwargs):
    '''
    Create a new secret with Name=name
    If you want SecretBytes instead of SecretString
    set value=None and specify SecretBytes in kwargs
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    
    resp = client.create_secret(Name=name, **kwargs)
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

def update(name, value, client=None, **kwargs):
    '''
    Update the value of a secret with SecretId=name
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    
    resp = client.put_secret_value(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

def delete(name, client=None, **kwargs):
    '''
    Schedule the secret with SecretId=name for deletion
    '''
    client = client or CLIENT
    resp = client.delete_secret(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

Here is a non-automated test that engages the live aws service.

In [ ]:
import uuid
def LIVE_test():
    secret_name = 'unittest/{}'.format(uuid.uuid4())

    ok, resp = create(secret_name, 'testvalue')
    #print(ok, resp)
    TESTCASE.assertTrue(ok)

    TESTCASE.assertEqual( get(secret_name), 'testvalue')

    ok, resp = update(secret_name, 'newvalue')
    #print(ok, resp)
    TESTCASE.assertTrue(ok)
    TESTCASE.assertEqual( get(secret_name), 'newvalue')

    ok, resp = delete(secret_name)
    #print(ok, resp)

    with TESTCASE.assertRaises(CLIENT.exceptions.InvalidRequestException):
        get(secret_name)
        
#LIVE_test()

In [ ]:
notebook2script(_nbpath)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/vera/vera/_nbdev.py'